In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, GRU, RepeatVector, TimeDistributed, Dense

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

PIVOTED_FILE = "pivoted_data.parquet"
ANOMALY_WINDOWS_FILE = "anomaly_windows.csv"
OUT_DIR = os.path.join("userData", "pivoted_paper_baseline")
os.makedirs(OUT_DIR, exist_ok=True)

SEQ_LEN = 1
ENCODER_UNITS = [16] * 7      
DECODER_UNITS = [16] * 7     
GRU_ACT = "relu"
USE_BOTTLENECK = True
BOTTLENECK_SIZE = 14

W_LONG, W_SHORT = 30, 2    
PAPER_THR = 0.9973          
ROUND_RES = "5T"             
EVENT_GAP_MINUTES = 15      
PCT_PROB = 0.05               

EPOCHS = 60
BATCH_SIZE = 64
LR = 1e-4

In [2]:
def read_parquet_smart(path, time_col="interval_start"):
    df = pd.read_parquet(path)
    if time_col in df.columns and not isinstance(df.index, pd.DatetimeIndex):
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
        df = df.sort_values(time_col).set_index(time_col)
    if isinstance(df.index, pd.DatetimeIndex) and df.index.year.max() < 1990:
        ns = df.index.view("int64")
        maxv = float(ns.max())
        if maxv < 1e12:
            scale = 1_000_000_000
        elif maxv < 1e15:
            scale = 1_000_000
        elif maxv < 1e18:
            scale = 1_000
        else:
            scale = 1
        df = df.copy()
        df.index = pd.to_datetime(ns * scale, unit="ns")
    return df

def add_time_features(index):
    t = pd.Series(index.view("int64") // 10**9, index=index)
    day = 24 * 3600
    week = 7 * day
    return pd.DataFrame({
        "sin_day": np.sin(2 * np.pi * (t % day) / day),
        "cos_day": np.cos(2 * np.pi * (t % day) / day),
        "sin_week": np.sin(2 * np.pi * (t % week) / week),
        "cos_week": np.cos(2 * np.pi * (t % week) / week),
    }, index=index).astype(np.float32)

def make_sequences(df_values, L=1):
    arr = df_values.values
    if L == 1:
        return arr[:, None, :]
    n = len(arr) - L + 1
    return np.stack([arr[i:i+L] for i in range(n)], axis=0)

In [3]:
NAB_PROFILES = {
    "standard": {"TP": 1.0, "FN": -1.0, "FP": -0.11, "TN": 0.0},
    "low_fp":  {"TP": 1.0, "FN": -1.0, "FP": -0.22, "TN": 0.0},
    "low_fn": {"TP": 1.0, "FN": -2.0, "FP": -0.11, "TN": 0.0}
}

def scaled_sigmoid(x, lo, hi, slope=5.0):
    v = 1.0 / (1.0 + np.exp(-slope * x))
    return lo + (hi - lo) * v

def tp_score(time_hit, start, end, tp_weight):
    time_hit = pd.Timestamp(time_hit)
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    L = (end - start).total_seconds()
    if L <= 0:
        return float(tp_weight)
    if time_hit <= start + pd.Timedelta(microseconds=1):
        return float(tp_weight)
    r = (time_hit - start).total_seconds() / L
    r = np.clip(r, 0.0, 1.0)
    x = 1.0 - 2.0 * r
    return float(scaled_sigmoid(x, 0.0, float(tp_weight)))

def fp_penalty(time_hit, windows, fp_weight):
    if not windows:
        return float(fp_weight)
    dists = []
    lens = []
    for (s, e) in windows:
        if s <= time_hit <= e:
            dists.append(0.0)
        elif time_hit < s:
            dists.append((s - time_hit).total_seconds())
        else:
            dists.append((time_hit - e).total_seconds())
        lens.append((e - s).total_seconds())
    d = float(min(dists))
    mean_len = float(np.mean(lens)) if len(lens) > 0 else 60.0
    z = d / max(mean_len, 1.0)
    val = scaled_sigmoid(z, float(fp_weight), 0.0)
    return float(min(0.0, val))

def read_gt_windows(csv_path, start_hint="start", end_hint="end"):
    gt = pd.read_csv(csv_path)
    s_cols = [c for c in gt.columns if start_hint in c.lower()]
    e_cols = [c for c in gt.columns if end_hint in c.lower()]
    if not s_cols or not e_cols:
        raise ValueError("Could not find start/end columns in anomaly_windows.csv")
    s_col = s_cols[0]
    e_col = e_cols[0]
    df = pd.DataFrame({
        "anomaly_start": pd.to_datetime(gt[s_col].astype(str).str.replace(r'([+-]\d{2}:?\d{2}|Z)$','', regex=True), errors="coerce"),
        "anomaly_end":   pd.to_datetime(gt[e_col].astype(str).str.replace(r'([+-]\d{2}:?\d{2}|Z)$','', regex=True), errors="coerce")
    }).dropna().reset_index(drop=True)
    return df

def mask_to_event_times(pred_mask, timestamps, event_gap_minutes=15, pct_prob=0.0):
    timestamps = np.asarray(timestamps)
    pm = np.asarray(pred_mask).astype(bool)
    if pct_prob > 0.0:
        cutoff = timestamps[0] + (timestamps[-1] - timestamps[0]) * pct_prob
        pm = pm & (timestamps >= cutoff)
    if not pm.any():
        return []
    shifted = np.concatenate(([False], pm[:-1]))
    edges = np.where(pm & (~shifted))[0]
    if len(edges) == 0:
        edges = np.array([np.where(pm)[0][0]])
    events = [pd.to_datetime(timestamps[edges[0]])]
    gap = pd.Timedelta(minutes=event_gap_minutes)
    for idx in edges[1:]:
        t = pd.to_datetime(timestamps[idx])
        if (t - events[-1]) >= gap:
            events.append(t)
    return events

def nab_score_from_mask(pred_mask, gt_windows_df, timestamps, profile="standard", event_gap_minutes=15, pct_prob=0.0):
    if profile not in NAB_PROFILES:
        raise ValueError("Unknown NAB profile: " + str(profile))
    prof = NAB_PROFILES[profile]
    TPw, FNw, FPw = prof["TP"], prof["FN"], prof["FP"]
    wins = [(pd.Timestamp(r["anomaly_start"]), pd.Timestamp(r["anomaly_end"])) for _, r in gt_windows_df.iterrows()]
    events = mask_to_event_times(pred_mask, timestamps, event_gap_minutes, pct_prob)
    score = 0.0
    TP = 0
    FP = 0
    used = [False] * len(wins)
    for ev in events:
        hit = -1
        for i, (s, e) in enumerate(wins):
            if (not used[i]) and (s <= ev <= e):
                hit = i
                break
        if hit >= 0:
            score += tp_score(ev, wins[hit][0], wins[hit][1], TPw)
            used[hit] = True
            TP += 1
        else:
            score += fp_penalty(ev, wins, FPw)
            FP += 1
    FN = sum(not u for u in used)
    score += FNw * FN
    perfect = TPw * len(wins)
    null = FNw * len(wins)
    if np.isclose(perfect, null):
        normalized = 0.0
    else:
        normalized = 100.0 * (score - null) / (perfect - null)
    return float(normalized), {"TP": int(TP), "FN": int(FN), "FP": int(FP)}

def likelihood_scores_from_s(err, W=30, Wp=2):
    T = len(err)
    L = np.zeros(T)
    eps = 1e-8
    for t in range(T):
        a = max(0, t - W + 1)
        b = t + 1
        a2 = max(0, t - Wp + 1)
        long_window = err[a:b]
        short_window = err[a2:b]
        mu = long_window.mean()
        std = long_window.std(ddof=1) if (b - a) > 1 else eps
        mu_s = short_window.mean()
        z = (mu_s - mu) / (std + eps)
        L[t] = norm.cdf(z)
    return L

In [4]:
print("Loading data from:", PIVOTED_FILE)
df = read_parquet_smart(PIVOTED_FILE, time_col="interval_start")

Loading data from: pivoted_data.parquet


In [5]:
include_patterns = [r"(5xx|_5\d\d_)", r"_count$"]
compiled = [re.compile(p, re.I) for p in include_patterns]
feature_cols = [c for c in df.columns if all(p.search(c) for p in compiled)]
print("Selected feature columns:", len(feature_cols))
if len(feature_cols) == 0:
    raise ValueError("No features matched the selection patterns.")

five_xx_cols = [c for c in df.columns if re.search(r"5xx|_5\d\d_", c, re.I) and c.endswith("_count")]
df["sum_5xx_count"] = df[five_xx_cols].fillna(0).sum(axis=1)

Selected feature columns: 2406


In [6]:
X = df[feature_cols].fillna(0).astype(np.float32)
X = pd.concat([X, add_time_features(X.index)], axis=1)

In [7]:
TRAIN_START, TRAIN_END = pd.Timestamp("2024-01-26"), pd.Timestamp("2024-02-29 23:59:59")
TEST_START, TEST_END = pd.Timestamp("2024-03-01"), pd.Timestamp("2024-05-31 23:59:59")

train_df = X[(X.index >= TRAIN_START) & (X.index <= TRAIN_END)].copy()
test_df  = X[(X.index >= TEST_START) & (X.index <= TEST_END)].copy()
print("Train rows:", len(train_df), "Test rows:", len(test_df))

Train rows: 10074 Test rows: 26487


In [8]:
gt_all = read_gt_windows(ANOMALY_WINDOWS_FILE)
gt_in_test = gt_all[(gt_all["anomaly_end"] >= test_df.index[0]) & (gt_all["anomaly_start"] <= test_df.index[-1])].reset_index(drop=True)
print("GT windows total:", len(gt_all), "GT inside test:", len(gt_in_test))

GT windows total: 25 GT inside test: 18


In [9]:
mask_train_anomalies = True
if mask_train_anomalies:
    train_idx = train_df.index
    keep_mask = np.ones(len(train_idx), dtype=bool)
    for s, e in gt_all.itertuples(index=False):
        keep_mask &= ~((train_idx >= s) & (train_idx <= e))
    removed = len(train_df) - keep_mask.sum()
    train_df = train_df.loc[keep_mask]
    print("Rows removed from training due to masking GT windows:", removed)

Rows removed from training due to masking GT windows: 100


In [10]:
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(train_df), index=train_df.index, columns=train_df.columns)
X_test_scaled = pd.DataFrame(scaler.transform(test_df), index=test_df.index, columns=test_df.columns)

Xtr_seq = make_sequences(X_train_scaled, SEQ_LEN)
Xte_seq = make_sequences(X_test_scaled, SEQ_LEN)
print("Train seq shape:", Xtr_seq.shape, "Test seq shape:", Xte_seq.shape)

Train seq shape: (9974, 1, 2410) Test seq shape: (26487, 1, 2410)


In [11]:
input_dim = Xtr_seq.shape[-1]
inp = Input(shape=(SEQ_LEN, input_dim), name="input")

x = inp

for u in ENCODER_UNITS[:-1]:
    x = GRU(u, return_sequences=True, activation=GRU_ACT)(x)

x = GRU(ENCODER_UNITS[-1], return_sequences=False, activation=GRU_ACT)(x)

if USE_BOTTLENECK:
    x = Dense(BOTTLENECK_SIZE, activation="relu", name="bottleneck")(x)
x = RepeatVector(SEQ_LEN)(x)
for u in DECODER_UNITS:
    x = GRU(u, return_sequences=True, activation=GRU_ACT)(x)

out = TimeDistributed(Dense(input_dim), name="reconstruction")(x)

model = Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(LR), loss="mse")
print("\nModel summary:")
model.summary()


Model summary:
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1, 2410)]         0         
_________________________________________________________________
gru (GRU)                    (None, 1, 16)             116544    
_________________________________________________________________
gru_1 (GRU)                  (None, 1, 16)             1632      
_________________________________________________________________
gru_2 (GRU)                  (None, 1, 16)             1632      
_________________________________________________________________
gru_3 (GRU)                  (None, 1, 16)             1632      
_________________________________________________________________
gru_4 (GRU)                  (None, 1, 16)             1632      
_________________________________________________________________
gru_5 (GRU)                  (None, 1, 16)   

In [12]:
history = model.fit(
    Xtr_seq, Xtr_seq,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_split=0.1,
    verbose=1
)
print("Training finished.")

Train on 8976 samples, validate on 998 samples
Epoch 1/60
8976/8976 [==============================] - 35s 4ms/sample - loss: 0.0013 - val_loss: 0.0010
Epoch 2/60
8976/8976 [==============================] - 2s 271us/sample - loss: 0.0012 - val_loss: 9.8035e-04
Epoch 3/60
8976/8976 [==============================] - 2s 271us/sample - loss: 0.0012 - val_loss: 9.1846e-04
Epoch 4/60
8976/8976 [==============================] - 2s 271us/sample - loss: 0.0011 - val_loss: 8.4708e-04
Epoch 5/60
8976/8976 [==============================] - 2s 272us/sample - loss: 9.7724e-04 - val_loss: 7.7162e-04
Epoch 6/60
8976/8976 [==============================] - 2s 277us/sample - loss: 8.8630e-04 - val_loss: 6.9754e-04
Epoch 7/60
8976/8976 [==============================] - 2s 273us/sample - loss: 8.0963e-04 - val_loss: 6.4046e-04
Epoch 8/60
8976/8976 [==============================] - 2s 265us/sample - loss: 7.6537e-04 - val_loss: 6.1952e-04
Epoch 9/60
8976/8976 [==============================] - 2s 265

In [13]:
recon = model.predict(Xte_seq, batch_size=max(1, BATCH_SIZE), verbose=0)
recon_flat = recon.reshape(recon.shape[0], recon.shape[2])    
true_flat = Xte_seq.reshape(Xte_seq.shape[0], Xte_seq.shape[2])
sq_err_each = (recon_flat - true_flat) ** 2
s_t = sq_err_each.mean(axis=1).astype(np.float32)  

L = likelihood_scores_from_s(s_t, W=W_LONG, Wp=W_SHORT)

In [14]:
timestamps = pd.to_datetime(X_test_scaled.index)[:len(L)]
try:
    timestamps = timestamps.tz_localize(None)
except Exception:
    pass
timestamps = timestamps.round(ROUND_RES)

In [15]:
gt_df = gt_in_test.copy()
gt_df["anomaly_start"] = pd.to_datetime(gt_df["anomaly_start"]).dt.tz_localize(None)
gt_df["anomaly_end"]   = pd.to_datetime(gt_df["anomaly_end"]).dt.tz_localize(None)
gt_df["start_rounded"] = gt_df["anomaly_start"].dt.round(ROUND_RES)
gt_df["end_rounded"]   = gt_df["anomaly_end"].dt.round(ROUND_RES)

t0, t1 = timestamps[0], timestamps[-1]
gt_df["start_clipped"] = gt_df["start_rounded"].clip(lower=t0, upper=t1)
gt_df["end_clipped"] = gt_df["end_rounded"].clip(lower=t0, upper=t1)
gt_df = gt_df[gt_df["start_clipped"] <= gt_df["end_clipped"]].reset_index(drop=True)

gt_series = pd.Series(0, index=timestamps)
for _, r in gt_df.iterrows():
    s, e = r["start_clipped"], r["end_clipped"]
    mask = (gt_series.index >= s) & (gt_series.index <= e)
    if mask.any():
        gt_series.loc[mask] = 1

scores_arr = np.asarray(L)
scores_df = pd.DataFrame({"timestamp": timestamps, "anomaly_score": scores_arr}).set_index("timestamp")
pred_series = (scores_df["anomaly_score"] >= PAPER_THR).astype(int)
pred_mask_bool = pred_series.values.astype(bool)

TP = int(((pred_series == 1) & (gt_series == 1)).sum())
FP = int(((pred_series == 1) & (gt_series == 0)).sum())
TN = int(((pred_series == 0) & (gt_series == 0)).sum())
FN = int(((pred_series == 0) & (gt_series == 1)).sum())
print("\nPer-timestamp counts:")
print(f"  TP={TP}  TN={TN}  FP={FP}  FN={FN}  total={len(pred_series)}")


Per-timestamp counts:
  TP=8  TN=25451  FP=163  FN=865  total=26487


In [16]:
norm_standard, counts_standard = nab_score_from_mask(pred_mask_bool, gt_df, timestamps,
                                                    profile="standard",
                                                    event_gap_minutes=EVENT_GAP_MINUTES,
                                                    pct_prob=PCT_PROB)
norm_lowfn, counts_lowfn = nab_score_from_mask(pred_mask_bool, gt_df, timestamps,
                                               profile="low_fn",
                                               event_gap_minutes=EVENT_GAP_MINUTES,
                                               pct_prob=PCT_PROB)
print("\nWindow-based normalized NAB scores:")
print(f"  Standard: {norm_standard:.2f} | event-counts: {counts_standard}")
print(f"  Low-FN  : {norm_lowfn:.2f} | event-counts: {counts_lowfn}")


Window-based normalized NAB scores:
  Standard: 17.00 | event-counts: {'TP': 4, 'FN': 14, 'FP': 121}
  Low-FN  : 18.74 | event-counts: {'TP': 4, 'FN': 14, 'FP': 121}


In [17]:
per_ts_path = os.path.join(OUT_DIR, f"per_timestamp_preds_thr_{str(PAPER_THR).replace('.', '')}.csv")
scores_df.assign(pred=pred_series.values, gt=gt_series.values).to_csv(per_ts_path)
events = mask_to_event_times(pred_mask_bool, scores_df.index, event_gap_minutes=EVENT_GAP_MINUTES, pct_prob=PCT_PROB)
ev_df = pd.DataFrame({"alert_time": pd.to_datetime(events)})
ev_path = os.path.join(OUT_DIR, f"baseline_alerts_thr_{str(PAPER_THR).replace('.', '')}.csv")
ev_df.to_csv(ev_path, index=False)
print(f"\nSaved per-timestamp preds to: {per_ts_path}")
print(f"Saved event alerts CSV to: {ev_path}")

try:
    plt.figure(figsize=(6, 4))
    counts = df["sum_5xx_count"].fillna(0).values
    if len(counts) > 0:
        min_x = max(1, counts.min() + 1)
        max_x = counts.max() + 1
        bins = np.logspace(max(0, np.log10(min_x) - 0.1), np.log10(max_x) + 0.1, 80)
        plt.hist(counts + 1e-9, bins=bins, histtype="step", linewidth=1.5)
        plt.xscale("log"); plt.yscale("log")
    plt.title("Distribution of 5XX counts (log-log)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "fig2_5xx_distribution.png"), dpi=150)
    plt.close()

    plt.figure(figsize=(14, 3))
    plt.plot(scores_df.index, scores_df["anomaly_score"], label="Lt (anomaly score)")
    plt.axhline(PAPER_THR, color="red", linestyle="--", label=f"thr={PAPER_THR}")
    for _, r in gt_df.iterrows():
        s, e = r["start_clipped"], r["end_clipped"]
        plt.axvspan(s, e, color="purple", alpha=0.12)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "diag_lt_with_gt.png"), dpi=150)
    plt.close()
    print("Saved diagnostic plots.")
except Exception as e:
    print("Plotting error (ignored):", e)


Saved per-timestamp preds to: userData/pivoted_paper_baseline/per_timestamp_preds_thr_09973.csv
Saved event alerts CSV to: userData/pivoted_paper_baseline/baseline_alerts_thr_09973.csv
Saved diagnostic plots.
